In [2]:
import pandas as pd
from matplotlib import pyplot as plt

ModuleNotFoundError: No module named 'pandas'

In [8]:
testdict = {'name': ['A', 'B', 'C'], 'int': ['1', '2', '3']}

df = pd.DataFrame.from_dict(testdict)

In [10]:
print(df)

  name int
0    A   1
1    B   2
2    C   3
